In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    setting = yaml.load(yaml_file)

path = make_path_dict(setting)

In [ ]:
signal = pd.read_table(path["feature_x_sample.context_signal.tsv"], index_col=0).fillna(
    0
)

In [ ]:
k_nmfcc = ccal.nmf_consensus_cluster_with_multiple_k(
    signal,
    setting["nmf_ks"],
    n_job=setting["max_n_job"],
    n_clustering=setting["nmf_n_clustering"],
    plot_df=signal.size < setting["plot_heat_map_max_size"],
    plot_w=signal.shape[0] < setting["plot_cluster_max_size"],
    plot_h=signal.shape[1] < setting["plot_cluster_max_size"],
    directory_path=path["nmf/"],
)

In [ ]:
for k in setting["nmf_ks"]:

    nmf_k_directory_path = "{}/{}/".format(path["nmf/"], k)

    w = pd.read_table("{}/w.tsv".format(nmf_k_directory_path), index_col=0)

    h = pd.read_table("{}/h.tsv".format(nmf_k_directory_path), index_col=0)

    w.columns.name = "Factor"

    h.columns.name = setting["sample_alias"]

    for w_or_h in ("w", "h"):

        if w_or_h == "w":

            df = w.T

        elif w_or_h == "h":

            df = h

        ccal.make_comparison_panel(
            df,
            df,
            axis=1,
            title="NMF K={} {}".format(k, w_or_h.title()),
            name_0=df.index.name,
            name_1=df.index.name,
        )